# Imports

In [3]:
import os
import sys
import cv2
import numpy as np
import time
import multiprocessing
import yaml
import baboon_tracking as bt
import baboon_tracking.registration
import baboon_tracking.foreground_extraction

# Config Setting

In [4]:
with open("config.yml", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        sys.exit()

# Get Video

In [8]:
cap = cv2.VideoCapture(config['input'])
assert cap.isOpened(), "Failed to open video {}".format(INPUT_VIDEO)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter(config['output'], cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (frame_width,frame_height))

cpus = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cpus)

# Set Up Tracker

#### IMPORTANT
Edit the host and port of bt.ImageStreamServer depending on where this code is run.  
If run in docker container, hostname should be 'rabbitmq' or whatever DNS name is assigned to the rabbit container.  
If run on local machine, hostname should be 'localhost'

In [ ]:
registration = bt.registration.ORB_RANSAC_Registration(config)
fg_extraction = bt.foreground_extraction.VariableBackgroundSub_ForegroundExtraction(config)

tracker = bt.BaboonTracker(config=config, registration=registration, foreground_extraction=fg_extraction, pool=pool)
server = bt.ImageStreamServer(host='rabbitmq', port='5672')

# Run Algorithm

In [ ]:
start = time.clock()
framecount = 1
# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # We need at least n frames to continue
        if (len(tracker.history_frames) < config['history_frames']):
            tracker.push_history_frame(gray)
            continue

        # returns list of tuples of (shifted frames, transformation matrix)
        shifted_history_frames = tracker.shift_history_frames(gray)

        # splits tuple list into two lists
        Ms = [f[1] for f in shifted_history_frames]
        shifted_history_frames = [f[0] for f in shifted_history_frames]

        # generates moving foreground mask
        moving_foreground = tracker.generate_motion_mask(gray, shifted_history_frames, Ms, framecount)

        # Send the resulting frame to rabbitmq
        server.imshow(moving_foreground)
        out.write(cv2.cvtColor(moving_foreground, cv2.COLOR_GRAY2BGR))

        tracker.push_history_frame(gray)

        curr_time = time.clock() - start

        print('curr_time: ' + str(curr_time))
        framecount = framecount + 1

    # Break the loop
    else:
        break

# When everything done, release the video capture object
cap.release()
out.release()

# Closes all the frames
cv2.destroyAllWindows()